In [1]:
#Inital imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lifetimes
from lifetimes.plotting import plot_period_transactions, plot_calibration_purchases_vs_holdout_purchases
from lifetimes import BetaGeoFitter
from datetime import timedelta
from datetime import datetime
from dateutil import parser
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
!ls data

OnlineRetail_2yrs.csv             lifetimes_object_df.csv
customer_data_base.zip            lifetimes_object_df_uncleaned.csv


In [3]:
#Load in our pre-prepared dataframes
lifetimes_object_df = pd.read_csv("data/lifetimes_object_df.csv")

### Preparation and evaluation functions

In [4]:
#Load in our data wrangler
from lifetimes.utils import summary_data_from_transaction_data
#Import holdout 
from lifetimes.utils import calibration_and_holdout_data

In [5]:
#class to return a calibration and holdout df
class df_ch():
    def __init__(self, eval_period=None,transaction_df=None,purchase_timestamp_col=None,customer_id_col=None,datetime_col=None,monetary_value_col=None):
        #initialized attributes
        self.eval_period = eval_period
        self.transaction_df = transaction_df
        self.purchase_timestamp_col=purchase_timestamp_col
        self.customer_id_col=customer_id_col
        self.datetime_col=datetime_col
        self.monetary_value_col=monetary_value_col
        #save off more attributes
        self.min_obs_date = parser.parse(transaction_df[purchase_timestamp_col].min())
        self.max_obs_date = parser.parse(transaction_df[purchase_timestamp_col].max())
        self.max_calib_date = self.max_obs_date - timedelta(days=eval_period)  
        self.calib_range_days = (self.max_calib_date - self.min_obs_date).days
    def df_ch_getdf(self):
        df = calibration_and_holdout_data(
        transactions = self.transaction_df, 
        customer_id_col=self.customer_id_col,
        datetime_col=self.datetime_col,
        monetary_value_col=self.monetary_value_col,
        calibration_period_end = self.max_calib_date, 
        observation_period_end = self.max_obs_date, 
        freq = "D")
        return df
        

In [6]:
#function to return a df of real and predicted transacitons in eval period
def bgf_real_v_pred_df(ch,bgf):
    # transactions in the observation period equals frequency_holdout + 1
    rfm_cal_holdout = pd.DataFrame()
    ch_df = ch.df_ch_getdf()
    rfm_cal_holdout["n_transactions_cal_real"]  = ch_df["frequency_cal"] + 1
    rfm_cal_holdout["n_transactions_holdout_real"]  = ch_df["frequency_holdout"] + 1
    # the predicted number of transactions
    rfm_cal_holdout["n_transactions_holdout_pred"] = bgf.predict(t=ch.eval_period, 
                                                    frequency=ch_df['frequency_cal'], 
                                                    recency=ch_df['recency_cal'], 
                                                    T=ch_df['T_cal'])
    return rfm_cal_holdout[["n_transactions_cal_real","n_transactions_holdout_real", "n_transactions_holdout_pred"]]

In [7]:
#function to capture RMSE for a BGF model
def bgf_rmse(ch,bgf):
    df_ch = ch.df_ch_getdf()
    df_ch["n_transactions_holdout_real"] = df_ch["frequency_holdout"] + 1
    y_true = df_ch["n_transactions_holdout_real"]
    y_pred = bgf.predict(t=ch.eval_period, frequency=df_ch['frequency_cal'],
                         recency=df_ch['recency_cal'],
                         T=df_ch['T_cal'])

    return mean_squared_error(y_true,y_pred)

In [8]:
#function to return predicted # transactions for given customer in evaluation period
def samp_cust_pred_trans(df_ch,sample_customer_id,eval_period):
    sample_customer = df_ch.loc[sample_customer_id]
    n_transactions_pred = bgf.predict(t=eval_period,
                                  frequency=sample_customer['frequency_cal'], 
                                  recency=sample_customer['recency_cal'], 
                                  T=sample_customer['T_cal'])
    return(n_transactions_pred)

In [9]:
#Dummy model function that predicts the same purchase rate in the observation period to continue through the eval period
def bgf_dummy_model(ch):
    df_ch = ch.df_ch_getdf()
    dummy_ch_df = pd.DataFrame()
    dummy_ch_df["purchases_per_period_cal"] = df_ch["frequency_cal"] + 1
    dummy_ch_df["purchases_per_period_cal"] = dummy_ch_df["purchases_per_period_cal"]/ch.calib_range_days
    dummy_ch_df["dummy_pred_purchases_holdout"] = dummy_ch_df["purchases_per_period_cal"] * ch.eval_period
    dummy_ch_df["actual_purchases_holdout"] = df_ch["frequency_holdout"] + 1
    return mean_squared_error(y_true=dummy_ch_df["dummy_pred_purchases_holdout"],y_pred=dummy_ch_df["actual_purchases_holdout"])

### RFM data exploration 

In [11]:
summary = summary_data_from_transaction_data(transactions=lifetimes_object_df,customer_id_col='customer_unique_id',
                                             datetime_col='order_purchase_timestamp',monetary_value_col='payment_value',observation_period_end='2018-10-17')
summary

,frequency,recency,T,monetary_value
customer_unique_id,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,160.0,0.0
0000b849f77a49e4a4ce2b2a4ca5be3f,0.0,0.0,163.0,0.0
0000f46a3911fa3c0805444483337064,0.0,0.0,586.0,0.0
0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,370.0,0.0
0004aac84e0df4da2b147fca70cf8255,0.0,0.0,337.0,0.0
...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,0.0,0.0,496.0,0.0
fffea47cd6d3cc0a88bd621562a9d061,0.0,0.0,311.0,0.0
ffff371b4d645b6ecea244b27531430a,0.0,0.0,617.0,0.0


In [12]:
#We have less than 3% of customers who are repeat purchasers
summary.frequency.value_counts(normalize=True)

0.0     0.978261
1.0     0.020097
2.0     0.001305
3.0     0.000189
4.0     0.000063
5.0     0.000053
6.0     0.000021
15.0    0.000011
Name: frequency, dtype: float64

In [137]:
summary.frequency.value_counts()

0.0     92924
1.0      1909
2.0       124
3.0        18
4.0         6
5.0         5
6.0         2
15.0        1
Name: frequency, dtype: int64

In [138]:
#create a list of repeat purchasers
repeat_purchaser_ids = list(summary[summary["frequency"] >= 1].index)
#subset our transaction data to just repeat purchasers
repeat_lifetimes_object_df = lifetimes_object_df[lifetimes_object_df["customer_unique_id"].isin(repeat_purchaser_ids) == True]
#Checking we removed the correct number of user id's
(len(lifetimes_object_df.customer_unique_id.unique()) - len(repeat_lifetimes_object_df.customer_unique_id.unique())) == summary.frequency.value_counts()[0]

True

In [14]:
repeat_purchaser_summary = summary_data_from_transaction_data(transactions=repeat_lifetimes_object_df,customer_id_col='customer_unique_id',
                                             datetime_col='order_purchase_timestamp',monetary_value_col='payment_value',observation_period_end='2018-10-17')


In [15]:
#The Gamma-Gamma submodel, assumes no relationship between the monetary value and the purchase frequency. 
#The correlation between the two vectors is close to 0 so we can use this model.
repeat_purchaser_summary[['monetary_value', 'frequency']].corr()

,monetary_value,frequency
monetary_value,1.00000,-0.00049
frequency,-0.00049,1.00000


In [16]:
#Load in our ch object with all data
ch_1 = df_ch(eval_period=240,transaction_df=lifetimes_object_df,purchase_timestamp_col='order_purchase_timestamp',
                   customer_id_col='customer_unique_id',datetime_col='order_purchase_timestamp',
                   monetary_value_col='payment_value')

In [17]:
#Load in our ch object with repeat purchaser data
ch_2 = df_ch(eval_period=240,transaction_df=repeat_lifetimes_object_df,purchase_timestamp_col='order_purchase_timestamp',
                   customer_id_col='customer_unique_id',datetime_col='order_purchase_timestamp',
                   monetary_value_col='payment_value')

In [150]:
#create an array to evaluate total purchases per customer in calib period
repeat_lifetimes_object_df_ts = repeat_lifetimes_object_df
repeat_lifetimes_object_df_ts["order_purchase_timestamp"] = pd.to_datetime(repeat_lifetimes_object_df["order_purchase_timestamp"])
purchases_in_cal = repeat_lifetimes_object_df_ts[repeat_lifetimes_object_df_ts["order_purchase_timestamp"]<=ch_2.max_calib_date]
purchases_in_cal = purchases_in_cal[purchases_in_cal["order_purchase_timestamp"]<=ch_2.max_calib_date].groupby("customer_unique_id").nunique()["order_purchase_timestamp"]
purchases_in_cal.sort_values()
purchases_in_cal = pd.DataFrame(purchases_in_cal)

In [172]:
#create an array to evaluate total purchases per customer in calib period
purchases_in_val = repeat_lifetimes_object_df_ts[repeat_lifetimes_object_df_ts["order_purchase_timestamp"]>ch_2.max_calib_date].groupby("customer_unique_id").nunique()["order_purchase_timestamp"]
purchases_in_val.sort_values()
purchases_in_val = pd.DataFrame(purchases_in_val)
purchases_in_val["order_purchase_timestamp"].sum()

2138

In [170]:
#Confirming that a 'frequency' of 0.00 in calibration period means total purchases in that period = 1
#Total calibration days with purchases = calibration frequency + 1
purch = []
for purchasers in df_ch_2["frequency_cal"]:    
    purch.append(purchasers+1)
sum(purch) == purchases_in_cal["order_purchase_timestamp"].sum()

True

In [177]:
#attempting on holdout group
df_ch_2["frequency_holdout"]
purch = []
zeros = 0
for purchasers in df_ch_2["frequency_holdout"]:    
    purch.append(purchasers+1)
    if purchasers == 0:
        zeros += 1
print(zeros)
sum(purch) == purchases_in_val["order_purchase_timestamp"].sum()

710


False

###  Dummy model
- Predict the same purchase as the observed in the calibration period

In [41]:
print(f' bgf dummy model RMSE w/ cleaned data: {bgf_dummy_model(ch_1)}')
print(f' bgf dummy model RMSE w/ cleaned data: {bgf_dummy_model(ch_2)}')

 bgf dummy model RMSE w/ cleaned data: 0.32524156574867047
 bgf dummy model RMSE w/ cleaned data: 1.2157071910136914


In [61]:
df_ch_2 = ch_2.df_ch_getdf()
df_ch_2.describe()

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
count,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.0
mean,0.599711,42.811416,162.373555,82.950796,0.535405,69.867975,240.0
std,0.626159,68.059084,98.159461,147.826952,0.631921,115.466449,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.0
25%,0.000000,0.000000,78.750000,0.000000,0.000000,0.000000,240.0
50%,1.000000,6.000000,156.000000,37.090000,0.000000,0.000000,240.0
75%,1.000000,60.000000,236.000000,113.650000,1.000000,102.817500,240.0
max,8.000000,455.000000,460.000000,1650.180000,7.000000,847.380000,240.0


In [63]:
ch_2.max_calib_date

datetime.datetime(2018, 1, 6, 0, 0)

5        2017-09-14
13       2018-03-07
32       2018-04-18
87       2018-05-16
115      2018-04-17
            ...    
98076    2018-01-24
98083    2017-06-28
98101    2018-06-20
98118    2018-05-15
98171    2018-01-24
Name: order_purchase_timestamp, Length: 4433, dtype: object

In [51]:
dummy_ch_df = pd.DataFrame()
dummy_ch_df["purchases_per_period_cal"] = df_ch_2["frequency_cal"] + 1
#533 is the calib range
dummy_ch_df["purchases_per_period_cal"] = dummy_ch_df["purchases_per_period_cal"]/533
dummy_ch_df["value_per_purchase_cal"] = df_ch_2["monetary_value_cal"] / (df_ch_2["frequency_cal"] + 1)
#240 is the eval range
dummy_ch_df["dummy_pred_purchases_holdout"] = dummy_ch_df["purchases_per_period_cal"] * 240
dummy_ch_df["actual_purchases_holdout"] = df_ch_2["frequency_holdout"] + 1
dummy_ch_df

,purchases_per_period_cal,value_per_purchase_cal,dummy_pred_purchases_holdout,actual_purchases_holdout
customer_unique_id,,,,
004288347e5e88a27ded2bb23747066c,0.001876,0.000000,0.450281,2.0
004b45ec5c64187465168251cd1c9c2f,0.001876,0.000000,0.450281,2.0
011b4adcd54683b480c4d841250a987f,0.001876,0.000000,0.450281,2.0
012452d40dafae4df401bced74cdb490,0.001876,0.000000,0.450281,2.0
013f4353d26bb05dc6652f1269458d8d,0.003752,82.020000,0.900563,1.0
...,...,...,...,...
fe59d5878cd80080edbd29b5a0a4e1cf,0.005629,13.200000,1.350844,1.0
fe81bb32c243a86b2f86fbf053fe6140,0.003752,31.890000,0.900563,4.0
fed519569d16e690df6f89cb99d4e682,0.001876,0.000000,0.450281,2.0


In [53]:
df_ch_2.describe()

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
count,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.0
mean,0.599711,42.811416,162.373555,82.950796,0.535405,69.867975,240.0
std,0.626159,68.059084,98.159461,147.826952,0.631921,115.466449,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.0
25%,0.000000,0.000000,78.750000,0.000000,0.000000,0.000000,240.0
50%,1.000000,6.000000,156.000000,37.090000,0.000000,0.000000,240.0
75%,1.000000,60.000000,236.000000,113.650000,1.000000,102.817500,240.0
max,8.000000,455.000000,460.000000,1650.180000,7.000000,847.380000,240.0


In [78]:
mean_squared_error(y_true=dummy_ch_df["dummy_pred_purchases_holdout"],y_pred=dummy_ch_df["actual_purchases_holdout"])

0.32524156574867047

###  1.0 BG/NBD model
Model features
- Uncleaned data

Model results
- Terrrible
- Our model is way off because we have such an imbalence of customers who never made a repeat purchase.

Next steps
- Drop customers with less than 2 orders

In [33]:
ch_1 = df_ch(eval_period=240,transaction_df=lifetimes_object_df_uncleaned,purchase_timestamp_col='order_purchase_timestamp',
                   customer_id_col='customer_unique_id',datetime_col='order_purchase_timestamp',
                   monetary_value_col='payment_value')

In [59]:
#import numpy as np
a = (ch_1.max_calib_date - ch_1.min_obs_date).days
a

533

In [26]:
df_ch_1 = ch_1.df_ch_getdf()
df_ch_1

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
customer_unique_id,,,,,,,
0000f46a3911fa3c0805444483337064,0.0,0.0,346.0,0.0,0.0,0.0,240.0
0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,130.0,0.0,0.0,0.0,240.0
0004aac84e0df4da2b147fca70cf8255,0.0,0.0,97.0,0.0,0.0,0.0,240.0
0005e1862207bf6ccc02e4228effd9a0,0.0,0.0,352.0,0.0,0.0,0.0,240.0
0006fdc98a402fceb4eb0ee528f6a8d4,0.0,0.0,216.0,0.0,0.0,0.0,240.0
...,...,...,...,...,...,...,...
fffbf87b7a1a6fa8b03f081c5f51a201,0.0,0.0,54.0,0.0,0.0,0.0,240.0
fffcf5a5ff07b0908bd4e2dbc735a684,0.0,0.0,256.0,0.0,0.0,0.0,240.0
fffea47cd6d3cc0a88bd621562a9d061,0.0,0.0,71.0,0.0,0.0,0.0,240.0


In [27]:
bgf_1 = BetaGeoFitter(penalizer_coef=0)

In [28]:
bgf_1.fit(
        frequency = df_ch_1["frequency_cal"], 
        recency = df_ch_1["recency_cal"], 
        T = df_ch_1["T_cal"],   
        weights = None,  
        verbose = True)

Optimization terminated successfully.
         Current function value: 0.069008
         Iterations: 63
         Function evaluations: 64
         Gradient evaluations: 64


<lifetimes.BetaGeoFitter: fitted with 55206 subjects, a: 1.41, alpha: 64.28, b: 0.24, r: 0.02>

In [30]:
bgf_1_rmse = bgf_rmse(ch_1,bgf_1)
bgf_1_rmse

1.009275270001345

In [79]:
bgf_real_v_pred_df(ch_1,bgf_1)

,n_transactions_cal_real,n_transactions_holdout_real,n_transactions_holdout_pred
customer_unique_id,,,
0000f46a3911fa3c0805444483337064,1.0,1.0,0.007896
0000f6ccb0745a6a4b88665a16c9f078,1.0,1.0,0.014077
0004aac84e0df4da2b147fca70cf8255,1.0,1.0,0.016051
0005e1862207bf6ccc02e4228effd9a0,1.0,1.0,0.007802
0006fdc98a402fceb4eb0ee528f6a8d4,1.0,1.0,0.010707
...,...,...,...
fffbf87b7a1a6fa8b03f081c5f51a201,1.0,1.0,0.019733
fffcf5a5ff07b0908bd4e2dbc735a684,1.0,1.0,0.009645
fffea47cd6d3cc0a88bd621562a9d061,1.0,1.0,0.018078
